# Retrieve RSS

In [1]:

!pip install feedparser requests importlib bs4 ffmpeg-python nltk soundfile numpy scipy firebase-admin google-cloud-firestore joblib
!pip install git+https://github.com/SWivid/F5-TTS.git


  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sgmllib3k-1.0.0-py3-none-any.whl
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 52.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for importlib: filename=importlib-1.0.4-py3-none-any.whl size=5850 sha256=f87a17060bc0c089e057dba83c598d5a4c5acfa19878fd4b9cae38cead905790
  Stored in directory: /Users/timur/Library/Caches/pip/wheels/40/41/c4/d925a53b7b7e75a65369e1b17f7bade00d7907ac5a7d74dc5f
Successfully built importlib
  Cloning https://github.com/SWivid/F5-TTS.git to /private/var/folde

In [2]:
from pyradiozilla import firebase

rz_firebase = firebase.Firebase()

In [3]:
from pyradiozilla import firebase

rz_author = firebase.RzAuthor(
    id= firebase.generate_url_safe_id('rss/weekly'),
    name='rss/weekly',
    description='Weekly rss summary scraper',        
    image=firebase.Blob(file_path="images/file-person.svg")
)
rz_author.upload_and_save(rz_firebase)
rz_channel = firebase.RzChannel(
    id= firebase.generate_url_safe_id('rss/python'),
    name='rss/python',    
    description='Latest updates about python programming language.',
    source_urls=['http://feeds.feedburner.com/PythonInsider'],
    image=firebase.Blob(file_path="images/rss.svg")
)
rz_channel.upload_and_save(rz_firebase)

AttributeError: module 'pyradiozilla.firebase' has no attribute 'generate_url_safe_id'

In [ ]:
from pyradiozilla import rss
from pyradiozilla import prompts
from pyradiozilla import ollama
from pyradiozilla import tts
from pyradiozilla import storage
import IPython
from pyradiozilla import firebase
from joblib import Parallel, delayed
import time
from typing import NamedTuple
from datetime import datetime

class Summary(NamedTuple):
    start_date: datetime
    prompt: str
    summary: str    
    directory: str

root_directory = storage.Directory("/media/timur/TESLA SSD")
print(f"Started scraping summary")    
entries = rss.RssScraper('http://feeds.feedburner.com/PythonInsider').parse()
summaryConfug = prompts.SummaryConfig(
    "English",
    prompts.SummaryLength.medium,
    prompts.SummaryTone.neutral,
    [prompts.SummaryFocus.key_points],
)

rss_prompts = prompts.RssWeeklySummaryPropmpt(entries, summaryConfug).get_prompts() 

print(f"Finished scraping summary")    
ollama_model = "gemma2:27b"

def create_summary(prompt: prompts.RssPrompt) -> Summary:
    print(f"Started rocessing weekly summary {prompt.start_date}")
    summary_directory = root_directory.get_timed_directory("run")
    summary_directory.get_file("prompt.txt").write_text(prompt.prompt)
    ollama_client = ollama.OllamaClient(ollama_model)       
    summary = ollama_client.generate(prompt.prompt)
    summary_directory.get_file("summary.txt").write_text(summary)
    print(f"Finished processing weekly summary {prompt.start_date}")    
    return Summary(
        start_date=prompt.start_date,
        prompt=prompt.prompt,
        summary=summary,
        directory=summary_directory.get_path()
    )
    
ollama.OllamaClient(ollama_model).unload_all_models()
    
print(f"Started summarizing")
        
rss_summaries = Parallel(n_jobs=4, backend='threading')(
    delayed(create_summary)(rss_prompt) for rss_prompt in rss_prompts
)

print(f"Started processing audio")

def create_audio(summary: Summary):
    print(f"Started processing audio {summary.start_date}")
    summary_directory = storage.Directory(summary.directory)
    audio = tts.BarkTTS.generate(summary.summary, summary_directory)
    rz_audio = firebase.RzAudio(
        id= firebase.generate_url_safe_uuid(),
        name='rss/python/weekly',
        description='Weekly rss summary about python programming language.',
        author_id=rz_author.id,
        channel_id=rz_channel.id,
        image=firebase.Blob(file_path="images/rss.svg"),
        audio=firebase.Blob(file_path=audio.m4a_file_path),
        topics=['Python', 'Programming']
    )
    summary_directory.get_file("rz_audio.txt").write_json(rz_audio)
    rz_audio.upload_and_save(rz_firebase)
    print(f"Finished processing audio {summary.start_date}")
    return summary    

IPython.display.display(rss_summaries)
    
rss_audios = Parallel(n_jobs=4, backend='threading')(
    delayed(create_audio)(
        Summary(
            start_date=rss_summary.start_date, 
            prompt=rss_summary.prompt, 
            summary=rss_summary.summary, 
            directory=rss_summary.directory
        )) for rss_summary in rss_summaries
)

IPython.display.display(rss_audios)

Started scraping summary
Finished scraping summary
Started summarizing
Started rocessing weekly summary 2024-10-14 00:00:00
Started rocessing weekly summary 2024-10-07 00:00:00
Started rocessing weekly summary 2024-09-30 00:00:00
Started rocessing weekly summary 2024-09-02 00:00:00
Finished processing weekly summary 2024-10-14 00:00:00
Started rocessing weekly summary 2024-08-05 00:00:00
Finished processing weekly summary 2024-09-02 00:00:00
Started rocessing weekly summary 2024-07-29 00:00:00
Finished processing weekly summary 2024-09-30 00:00:00
Started rocessing weekly summary 2024-07-15 00:00:00
Finished processing weekly summary 2024-10-07 00:00:00
Started rocessing weekly summary 2024-06-24 00:00:00
Finished processing weekly summary 2024-08-05 00:00:00
Started rocessing weekly summary 2024-06-03 00:00:00
Finished processing weekly summary 2024-07-15 00:00:00
Started rocessing weekly summary 2024-05-06 00:00:00
Finished processing weekly summary 2024-06-24 00:00:00
Started rocess

[Summary(start_date=datetime.datetime(2024, 10, 14, 0, 0), prompt="Provide a neutral summary in English that focuses on key points. The summary should be medium in length.\n\nNews from 2024-10-14 to 2024-10-20:\n\nPython 3.14.0 alpha 1 is now available: It's now time for a new alpha of a new version of Python!\nhttps://www.python.org/downloads/release/python-3140a1/\nThis is an early developer preview of Python\n3.14\nMajor\nnew features of the 3.14 series, compared to 3.13\nPython 3.14 is still in development. This release, 3.14.0a1 is the\nfirst of seven planned alpha releases.\nAlpha releases are intended to make it easier to test the current\nstate of new features and bug fixes and to test the release process.\nDuring the alpha phase, features may be added up until the start of\nthe beta phase (2025-05-06) and, if necessary, may be modified or\ndeleted up until the release candidate phase (2025-07-22). Please keep\nin mind that this is a preview release and its use is\nnot recommen

Started processing audio 2024-10-14 00:00:00
Started processing audio 2024-10-07 00:00:00
Started processing audio 2024-09-30 00:00:00
Started processing audio 2024-09-02 00:00:00


/home/timur/radiozilla/.venv/lib/python3.12/site-packages/bark/generation.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_locati